<a href="https://colab.research.google.com/github/alkadhi/scs/blob/main/CSCI_B363_BLAST_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Basic Local Alignment Search Tool (BLAST)**

Algorithm steps:
- Protein Word List (Neighborhood words)
  - Keep words that meet or exceed a certain threshold value, discard all others
- Scanning for word "hits":
  - For each word, create a lookup table (hash table) where each word is mapped to all occurrences of that word in the database sequence (by index)
  - Scanning the database part is done using the KMP exact pattern matching algorithm (from the lecture)
- Extending "hits" based on score obtained from scoring matrix
  - Extension process is terminated in one direction when the segment pair score (between query database sequences) falls a certain distance below the best score found for shorter extensions

- Optional: Account for gaps using Affine gap penalty

- May need to access NCBI protein database for meaningful tests
- Random protein sequence generator (for testing purposes): https://www.bioinformatics.org/sms2/random_protein.html
- Protein sequence is fetched from the NCBI protein database using the python biopython module: To run this program locally, install the biopython module using 'pip install biopython'
- Chain-B insulin-like receptor, Drosophila melanogaster (fruit fly)
https://www.ncbi.nlm.nih.gov/protein/9EF4_B

- Research papers on this topic:

Stephen F Altschul, Warren Gish, Webb Miller, Eugene W Myers, and David J
Lipman. Basic local alignment search tool. Journal of molecular biology, 215(3):403–
410, 1990.

https://www.biostat.wisc.edu/bmi576/papers/blast.pdf


Samuel Karlin and Stephen F Altschul. Methods for assessing the statistical
significance of molecular sequence features by using general scoring
schemes. Proceedings of the National Academy of Sciences, 87(6):2264–2268, 1990.

https://web.stanford.edu/class/sbio228/public/readings/Bioinformatics_I_Lecture6/Karlin_PNAS_90_Statistical_significance_alignment.pdf

Additional BLAST paper:

https://link.springer.com/article/10.1186/gb-2001-2-10-reviews2002





In [ ]:
# Install BioPython using pip
!pip install biopython

In [ ]:
from Bio import Entrez, SeqIO
import io

"""
Use the biopython module to get real protein database sequences from the NCBI protein database.
"""
def getNCBIProteinDatabase():

    # Access to NCBI database requires email address
    Entrez.email = "kaihara@iu.edu"

    # Accession number for insulin-like Chain-B receptor, Drosophila melanogaster (fruit fly)
    accessionID = "9EF4_B"

    # Retrieve the record from the protein database
    # Params:
    # db = 'protein' for protein sequences
    # rettype = 'fasta' to get the sequence in FASTA format
    # retmode = 'text' for text output

    try:
        handle = Entrez.efetch(db = "protein", id = accessionID, rettype = "fasta", retmode="text")
        record = handle.read()
        handle.close()

        # Print the raw FASTA record to verify
        print("Raw FASTA Record: ")
        print(record)

    except Exception as e:
        print(f"An error occurred during Entrez.efetch: {e}")
        record = None

    if record:
        # Use io.StringIO to treat the string
        record_handle = io.StringIO(record)

        # SeqIO.read parses the FASTA format record
        seq_record = SeqIO.read(record_handle, "fasta")

        # Extract the sequence object (Seq) and convert it to a standard Python string
        protein_sequence_string = str(seq_record.seq)

        # Print the resulting string (for testing)
        print(f"Sequence Accession ID: {seq_record.id}")
        print(f"Sequence Description: {seq_record.description}")
        print(f"Length of Sequence: {len(seq_record.seq)} residues")
        print("\n--- Pure Protein Sequence String ---")
        print(protein_sequence_string)
        print("--------------------------------------")

        return protein_sequence_string

    # If the record could not be found (or other issue), then just return None
    return None

In [ ]:
"""
Function for generating initial words from the protein sequence
ie. Query sequence: LIAWHCMPNAAA
initial words (length = 3): LIA, IAW, AWH, WHC, HCM, CMP, MPN, PNA, NAA, AAA
querySequence: Query protein sequence to use
wordLength: Length of each word to generate
Return: A list containing all initial words of length wordLength
"""
def generateInitialWords(querySequence, wordLength):
    allWords = []
    for i in range(len(querySequence) - wordLength + 1):
        # include the index i at there the initial word is generated
        allWords.append((i, querySequence[i: i + wordLength]))

    return allWords

In [ ]:
"""
Testing function for generateInitialWords()
"""
def testGenerateInitialWords():
  querySequence = "LIAWHCMPNAAA"
  wordLength = 3
  words = generateInitialWords(querySequence, wordLength)
  for word in words:
    print(word)

In [ ]:
import itertools

# TODO maybe change this to a more manual approach, that doesn't use itertools

"""
Generate all of the possible amino acid sequences given a sequence length of k
itertools: Python module to help with generating all possible amino acid character combinations
aminoAcids: All amino acids
k: Length of each amino acid sequence
Return: A list containing all possible amino acid sequences
"""
def generateAminoAcidSequences(aminoAcids, k):
  # Iterates over all of the possible combinations of aminoAcids characters, with each word length equal to k
  sequencesIterator = itertools.product(aminoAcids, repeat=k)
  # Converts from list to a string
  allSequences = list(map("".join, sequencesIterator))

  return allSequences

In [ ]:
#we can see if this works okay during our meeting

def generateAminoAcidSequences_manual(aminoAcids, k):
    # Start with empty prefix
    sequences = [""]

    for i in range(k):
        new_sequences = []
        for prefix in sequences:
            for aa in aminoAcids:
                new_sequences.append(prefix + aa)
        sequences = new_sequences

    return sequences

def testGenerateAminoAcidSequences_manual():
  k = 3
  aminoAcids = aminoAcids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
  lst = generateAminoAcidSequences(aminoAcids, k)
  print("Number of sequences: " + str(len(lst)))
  for l in lst:
    print(l)
testGenerateAminoAcidSequences_manual()


Number of sequences: 8000
AAA
AAR
AAN
AAD
AAC
AAQ
AAE
AAG
AAH
AAI
AAL
AAK
AAM
AAF
AAP
AAS
AAT
AAW
AAY
AAV
ARA
ARR
ARN
ARD
ARC
ARQ
ARE
ARG
ARH
ARI
ARL
ARK
ARM
ARF
ARP
ARS
ART
ARW
ARY
ARV
ANA
ANR
ANN
AND
ANC
ANQ
ANE
ANG
ANH
ANI
ANL
ANK
ANM
ANF
ANP
ANS
ANT
ANW
ANY
ANV
ADA
ADR
ADN
ADD
ADC
ADQ
ADE
ADG
ADH
ADI
ADL
ADK
ADM
ADF
ADP
ADS
ADT
ADW
ADY
ADV
ACA
ACR
ACN
ACD
ACC
ACQ
ACE
ACG
ACH
ACI
ACL
ACK
ACM
ACF
ACP
ACS
ACT
ACW
ACY
ACV
AQA
AQR
AQN
AQD
AQC
AQQ
AQE
AQG
AQH
AQI
AQL
AQK
AQM
AQF
AQP
AQS
AQT
AQW
AQY
AQV
AEA
AER
AEN
AED
AEC
AEQ
AEE
AEG
AEH
AEI
AEL
AEK
AEM
AEF
AEP
AES
AET
AEW
AEY
AEV
AGA
AGR
AGN
AGD
AGC
AGQ
AGE
AGG
AGH
AGI
AGL
AGK
AGM
AGF
AGP
AGS
AGT
AGW
AGY
AGV
AHA
AHR
AHN
AHD
AHC
AHQ
AHE
AHG
AHH
AHI
AHL
AHK
AHM
AHF
AHP
AHS
AHT
AHW
AHY
AHV
AIA
AIR
AIN
AID
AIC
AIQ
AIE
AIG
AIH
AII
AIL
AIK
AIM
AIF
AIP
AIS
AIT
AIW
AIY
AIV
ALA
ALR
ALN
ALD
ALC
ALQ
ALE
ALG
ALH
ALI
ALL
ALK
ALM
ALF
ALP
ALS
ALT
ALW
ALY
ALV
AKA
AKR
AKN
AKD
AKC
AKQ
AKE
AKG
AKH
AKI
AKL
AKK
AKM
AKF
AKP
AKS
AKT
AKW
AKY
AKV
AMA
AMR
AMN
AM

In [ ]:
"""
Testing function for generateAminoAcidSequences
"""
def testGenerateAminoAcidSequences():
  k = 3
  aminoAcids = aminoAcids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']
  lst = generateAminoAcidSequences(aminoAcids, k)
  print("Number of sequences: " + str(len(lst)))
  for l in lst:
    print(l)
testGenerateAminoAcidSequences()

Number of sequences: 8000
AAA
AAR
AAN
AAD
AAC
AAQ
AAE
AAG
AAH
AAI
AAL
AAK
AAM
AAF
AAP
AAS
AAT
AAW
AAY
AAV
ARA
ARR
ARN
ARD
ARC
ARQ
ARE
ARG
ARH
ARI
ARL
ARK
ARM
ARF
ARP
ARS
ART
ARW
ARY
ARV
ANA
ANR
ANN
AND
ANC
ANQ
ANE
ANG
ANH
ANI
ANL
ANK
ANM
ANF
ANP
ANS
ANT
ANW
ANY
ANV
ADA
ADR
ADN
ADD
ADC
ADQ
ADE
ADG
ADH
ADI
ADL
ADK
ADM
ADF
ADP
ADS
ADT
ADW
ADY
ADV
ACA
ACR
ACN
ACD
ACC
ACQ
ACE
ACG
ACH
ACI
ACL
ACK
ACM
ACF
ACP
ACS
ACT
ACW
ACY
ACV
AQA
AQR
AQN
AQD
AQC
AQQ
AQE
AQG
AQH
AQI
AQL
AQK
AQM
AQF
AQP
AQS
AQT
AQW
AQY
AQV
AEA
AER
AEN
AED
AEC
AEQ
AEE
AEG
AEH
AEI
AEL
AEK
AEM
AEF
AEP
AES
AET
AEW
AEY
AEV
AGA
AGR
AGN
AGD
AGC
AGQ
AGE
AGG
AGH
AGI
AGL
AGK
AGM
AGF
AGP
AGS
AGT
AGW
AGY
AGV
AHA
AHR
AHN
AHD
AHC
AHQ
AHE
AHG
AHH
AHI
AHL
AHK
AHM
AHF
AHP
AHS
AHT
AHW
AHY
AHV
AIA
AIR
AIN
AID
AIC
AIQ
AIE
AIG
AIH
AII
AIL
AIK
AIM
AIF
AIP
AIS
AIT
AIW
AIY
AIV
ALA
ALR
ALN
ALD
ALC
ALQ
ALE
ALG
ALH
ALI
ALL
ALK
ALM
ALF
ALP
ALS
ALT
ALW
ALY
ALV
AKA
AKR
AKN
AKD
AKC
AKQ
AKE
AKG
AKH
AKI
AKL
AKK
AKM
AKF
AKP
AKS
AKT
AKW
AKY
AKV
AMA
AMR
AMN
AM

In [ ]:
"""
Function for generating all neighborhood words to a specific sequence that meet above a certain threshold value
aminoAcids: All amino acids
BLOSUM62: BLOSUM62 scoring matrix
subsequence: Base subsequence to use for generating neighborhood values
threshold: Threshold value to evaluate each neighborhood word to keep or throw out
Return: List of all neighborhood words associated with a particular sequence, and the numeric scores associated with each word
"""
def generateNeighborhoodWords(aminoAcids, BLOSUM62, subsequence, threshold):
    # Store all neighborhood words that meet or exceed threshold value
    validNeighborhoodWords = []

    # Iterate over all of the posible neighborhood word combinations
    allAACombinations = generateAminoAcidSequences_manual(aminoAcids, len(subsequence))

    # For each combination, calculate the score, and compare against the threshold
    for combination in allAACombinations:
        currentScore = 0
        for i in range(len(subsequence)):
            currentScore += BLOSUM62[subsequence[i]][combination[i]]

        # If the score for the current combination >= threshold, add to validNeighborhoodWords
        if currentScore >= threshold:
            validNeighborhoodWords.append((currentScore, combination))

        # Else, discard and move to next iteration

    return validNeighborhoodWords

In [ ]:
def generateNeighborhoodWords(aminoAcids, BLOSUM62, subsequence, threshold):
    validNeighborhoodWords = []
    k = len(subsequence)

    allAACombinations = generateAminoAcidSequences_manual(aminoAcids, k)

    for combination in allAACombinations:
      # Storing a list of scores for each character by character alignment
        positionScores = []
        # Storing a sum of the score for all characters in the current combination
        currentScore = 0

        for i in range(k):
            score_ij = BLOSUM62[subsequence[i]][combination[i]]
            positionScores.append(score_ij)
            currentScore += score_ij

        if currentScore >= threshold:
            validNeighborhoodWords.append((currentScore, combination, positionScores))

    return validNeighborhoodWords


In [ ]:
"""
Testing function for generateNeighborhoodWords
"""

def testGenerateNeighborhoodWords():

  aminoAcids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

  # Python dictionary (Hash table) containing the scoring values between two amino acids
  BLOSUM62 = {
    'A': {'A': 4, 'R': -1, 'N': -2, 'D': -2, 'C': 0, 'Q': -1, 'E': -1, 'G': 0, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 0, 'W': -3, 'Y': -2, 'V': 0},
    'R': {'A': -1, 'R': 5, 'N': 0, 'D': -2, 'C': -3, 'Q': 1, 'E': 0, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 2, 'M': -1, 'F': -3, 'P': -2, 'S': -1, 'T': -1, 'W': -3, 'Y': -2, 'V': -3},
    'N': {'A': -2, 'R': 0, 'N': 6, 'D': 1, 'C': -3, 'Q': 0, 'E': 0, 'G': 0, 'H': 1, 'I': -3, 'L': -3, 'K': 0, 'M': -2, 'F': -3, 'P': -2, 'S': 1, 'T': 0, 'W': -4, 'Y': -2, 'V': -3},
    'D': {'A': -2, 'R': -2, 'N': 1, 'D': 6, 'C': -3, 'Q': 0, 'E': 2, 'G': -1, 'H': -1, 'I': -3, 'L': -4, 'K': -1, 'M': -3, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -4, 'Y': -3, 'V': -3},
    'C': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': 9, 'Q': -3, 'E': -4, 'G': -3, 'H': -3, 'I': -1, 'L': -1, 'K': -3, 'M': -1, 'F': -2, 'P': -3, 'S': -1, 'T': -1, 'W': -2, 'Y': -2, 'V': -1},
    'Q': {'A': -1, 'R': 1, 'N': 0, 'D': 0, 'C': -3, 'Q': 5, 'E': 2, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 1, 'M': 0, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -2, 'Y': -1, 'V': -2},
    'E': {'A': -1, 'R': 0, 'N': 0, 'D': 2, 'C': -4, 'Q': 2, 'E': 5, 'G': -2, 'H': 0, 'I': -3, 'L': -3, 'K': 1, 'M': -2, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
    'G': {'A': 0, 'R': -2, 'N': 0, 'D': -1, 'C': -3, 'Q': -2, 'E': -2, 'G': 6, 'H': -2, 'I': -4, 'L': -4, 'K': -2, 'M': -3, 'F': -3, 'P': -2, 'S': 0, 'T': -2, 'W': -2, 'Y': -3, 'V': -3},
    'H': {'A': -2, 'R': 0, 'N': 1, 'D': -1, 'C': -3, 'Q': 0, 'E': 0, 'G': -2, 'H': 8, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -1, 'P': -2, 'S': -1, 'T': -2, 'W': -2, 'Y': 2, 'V': -3},
    'I': {'A': -1, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -3, 'E': -3, 'G': -4, 'H': -3, 'I': 4, 'L': 2, 'K': -3, 'M': 1, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -3, 'Y': -1, 'V': 3},
    'L': {'A': -1, 'R': -2, 'N': -3, 'D': -4, 'C': -1, 'Q': -2, 'E': -3, 'G': -4, 'H': -3, 'I': 2, 'L': 4, 'K': -2, 'M': 2, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -2, 'Y': -1, 'V': 1},
    'K': {'A': -1, 'R': 2, 'N': 0, 'D': -1, 'C': -3, 'Q': 1, 'E': 1, 'G': -2, 'H': -1, 'I': -3, 'L': -2, 'K': 5, 'M': -1, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -3},
    'M': {'A': -1, 'R': -1, 'N': -2, 'D': -3, 'C': -1, 'Q': 0, 'E': -2, 'G': -3, 'H': -2, 'I': 1, 'L': 2, 'K': -1, 'M': 5, 'F': 0, 'P': -2, 'S': -1, 'T': -1, 'W': -1, 'Y': -1, 'V': 1},
    'F': {'A': -2, 'R': -3, 'N': -3, 'D': -3, 'C': -2, 'Q': -3, 'E': -3, 'G': -3, 'H': -1, 'I': 0, 'L': 0, 'K': -3, 'M': 0, 'F': 6, 'P': -4, 'S': -2, 'T': -2, 'W': 1, 'Y': 3, 'V': -1},
    'P': {'A': -1, 'R': -2, 'N': -2, 'D': -1, 'C': -3, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -4, 'P': 7, 'S': -1, 'T': -1, 'W': -4, 'Y': -3, 'V': -2},
    'S': {'A': 1, 'R': -1, 'N': 1, 'D': 0, 'C': -1, 'Q': 0, 'E': 0, 'G': 0, 'H': -1, 'I': -2, 'L': -2, 'K': 0, 'M': -1, 'F': -2, 'P': -1, 'S': 4, 'T': 1, 'W': -3, 'Y': -2, 'V': -2},
    'T': {'A': 0, 'R': -1, 'N': 0, 'D': -1, 'C': -1, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 5, 'W': -2, 'Y': -2, 'V': 0},
    'W': {'A': -3, 'R': -3, 'N': -4, 'D': -4, 'C': -2, 'Q': -2, 'E': -3, 'G': -2, 'H': -2, 'I': -3, 'L': -2, 'K': -3, 'M': -1, 'F': 1, 'P': -4, 'S': -3, 'T': -2, 'W': 11, 'Y': 2, 'V': -3},
    'Y': {'A': -2, 'R': -2, 'N': -2, 'D': -3, 'C': -2, 'Q': -1, 'E': -2, 'G': -3, 'H': 2, 'I': -1, 'L': -1, 'K': -2, 'M': -1, 'F': 3, 'P': -3, 'S': -2, 'T': -2, 'W': 2, 'Y': 7, 'V': -1},
    'V': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -2, 'E': -2, 'G': -3, 'H': -3, 'I': 3, 'L': 1, 'K': -3, 'M': 1, 'F': -1, 'P': -2, 'S': -2, 'T': 0, 'W': -3, 'Y': -1, 'V': 4}
}

  subsequence = "LIA"
  # 11 is an arbitrary value that is used as the default in some BLAST implementations
  threshold = 11

  neighborhoodWords = generateNeighborhoodWords(aminoAcids, BLOSUM62, subsequence, threshold)
  for word in neighborhoodWords:
    print(word)

In [ ]:
"""
Find the initial alignment between a word in the query sequence and the database sequence, works with one subsequence at a time
If there is a "hit" (match), then get the index at where this match occurred
Use one of the exact pattern matching algorithms from class, KMP algorithm is best for time and space complexity
subsequence: word subsequence (neighborhood word to search in database)
databaseSequence: database sequence to search into
Return: A list of all "hits" (indexes) from the database sequence
"""

def findInitialPatternMatch(subsequence, databaseSequence):

    def computeFailureArray(subsequence, m, failureArray):
        # length of the previous longest prefix suffix
        length = 0
        i = 1
        # calculate failureArray[i] for i = 1 to m - 1
        while i < m:
            if subsequence[i] == subsequence[length]:
                length += 1
                failureArray[i] = length
                i += 1

            else:
                # Jumping-back step in failureArray
                if length != 0:
                    length = failureArray[length - 1]

                else:
                    failureArray[i] = 0
                    i += 1

    # list to store all "hit" indexes
    indexList = []

    # Use KMP algorithm for exact pattern matching
    m = len(subsequence)
    n = len(databaseSequence)

    # create lps[] that will hold the longest prefix suffix values for pattern
    failureArray = [0] * m
    # index for subsequence
    j = 0

    computeFailureArray(subsequence, m, failureArray)
    i = 0 # index for databaseSequence[]
    while i < n:
        if subsequence[j] == databaseSequence[i]:
            i += 1
            j += 1

        if j == m:
            # print("Found pattern at index: " + str(i - j))
            indexList.append(i - j)
            j = failureArray[j - 1]

        # mismatch after j matches
        elif i < n and subsequence[j] != databaseSequence[i]:
            # Do not match failureArray[0..failureArray[j - 1]] characters,
            # they will match anyway
            if j != 0:
                # important KMP algorithm step for saving time
                j = failureArray[j - 1]
            else:
                i += 1

    return indexList

In [ ]:
"""
Testing function for findInitialPatternMatch
"""
def testFindInitialPatternMatch():

    databaseSequenceTest = "LIAATGLILIALIA"
    subsequenceTest = "LIA"
    indexListTest = findInitialPatternMatch(subsequenceTest, databaseSequenceTest)

    for index in indexListTest:
        print(index)

In [ ]:
"""
Once an alignment is found between the query word and a database subsequence, try and extend
the sequence in both directions
word: seed word to use to extend the alignment from
wordQueryStartIndex: index in the query sequence where the word 'starts' from
wordDataStartIndex: index in the database sequence where the word 'starts' from
extensionThreshold: minimum threshold value that the extension should be above
querySequence: query sequence to search from
databaseSequence: database sequence to search into
BLOSUM62: BLOSUM62 matrix to use for pair scoring
Return: A list containing the extension of the sequence (aligned section from the database sequence), and the score for that extension
"""
def extendAlignment(word, wordScore, wordQueryStartIndex, wordDataStartIndex, extensionThreshold, querySequence, databaseSequence, BLOSUM62):

    # Start with just the word, but final will be extensionLeft + word + extensionRight
    sequenceAlignment = word
    extensionScore = wordScore

    # Extend the alignment to the right, until it falls below a predefined threshold (0 for now)
    dataIndexRight = wordDataStartIndex + len(word)
    queryIndexRight = wordQueryStartIndex + len(word)

    while queryIndexRight < len(querySequence) and dataIndexRight < len(databaseSequence):
        currentQueryChar = querySequence[queryIndexRight]
        currentDataChar = databaseSequence[dataIndexRight]
        currentCharAlignmentScore = BLOSUM62[currentQueryChar][currentDataChar]

        # Check if adding the current character keeps the extensionScore above the extensionThreshold
        if extensionScore + currentCharAlignmentScore < extensionThreshold:
            break

        extensionScore += currentCharAlignmentScore
        sequenceAlignment += currentDataChar
        queryIndexRight += 1
        dataIndexRight += 1

    # Extend the alignment to the left, until it falls below a predefined threshold (0 for now)
    dataIndexLeft = wordDataStartIndex - 1
    queryIndexLeft = wordQueryStartIndex - 1

    while queryIndexLeft >= 0 and dataIndexLeft >= 0:
        currentQueryChar = querySequence[queryIndexLeft]
        currentDataChar = databaseSequence[dataIndexLeft]
        currentCharAlignmentScore = BLOSUM62[currentQueryChar][currentDataChar]

        # Check if adding the current character keeps the extensionScore above the extensionThreshold
        if extensionScore + currentCharAlignmentScore < extensionThreshold:
            break

        extensionScore += currentCharAlignmentScore
        sequenceAlignment = currentDataChar + sequenceAlignment
        queryIndexLeft -= 1
        dataIndexLeft -= 1

    return [sequenceAlignment, extensionScore]

In [ ]:
"""
Testing function for extendAlignment()
"""
def testExtendAlignment():
    # Example sequences
    querySequence = "LIAWHCMPNAAA"
    databaseSequence = "MQLIAWHCMPNAAAKK"

    word = "WHC"

    # Find where the word starts in each sequence
    wordQueryStartIndex = querySequence.index(word)
    wordDataStartIndex = databaseSequence.index(word)

    BLOSUM62 = {
    'A': {'A': 4, 'R': -1, 'N': -2, 'D': -2, 'C': 0, 'Q': -1, 'E': -1, 'G': 0, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 0, 'W': -3, 'Y': -2, 'V': 0},
    'R': {'A': -1, 'R': 5, 'N': 0, 'D': -2, 'C': -3, 'Q': 1, 'E': 0, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 2, 'M': -1, 'F': -3, 'P': -2, 'S': -1, 'T': -1, 'W': -3, 'Y': -2, 'V': -3},
    'N': {'A': -2, 'R': 0, 'N': 6, 'D': 1, 'C': -3, 'Q': 0, 'E': 0, 'G': 0, 'H': 1, 'I': -3, 'L': -3, 'K': 0, 'M': -2, 'F': -3, 'P': -2, 'S': 1, 'T': 0, 'W': -4, 'Y': -2, 'V': -3},
    'D': {'A': -2, 'R': -2, 'N': 1, 'D': 6, 'C': -3, 'Q': 0, 'E': 2, 'G': -1, 'H': -1, 'I': -3, 'L': -4, 'K': -1, 'M': -3, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -4, 'Y': -3, 'V': -3},
    'C': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': 9, 'Q': -3, 'E': -4, 'G': -3, 'H': -3, 'I': -1, 'L': -1, 'K': -3, 'M': -1, 'F': -2, 'P': -3, 'S': -1, 'T': -1, 'W': -2, 'Y': -2, 'V': -1},
    'Q': {'A': -1, 'R': 1, 'N': 0, 'D': 0, 'C': -3, 'Q': 5, 'E': 2, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 1, 'M': 0, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -2, 'Y': -1, 'V': -2},
    'E': {'A': -1, 'R': 0, 'N': 0, 'D': 2, 'C': -4, 'Q': 2, 'E': 5, 'G': -2, 'H': 0, 'I': -3, 'L': -3, 'K': 1, 'M': -2, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
    'G': {'A': 0, 'R': -2, 'N': 0, 'D': -1, 'C': -3, 'Q': -2, 'E': -2, 'G': 6, 'H': -2, 'I': -4, 'L': -4, 'K': -2, 'M': -3, 'F': -3, 'P': -2, 'S': 0, 'T': -2, 'W': -2, 'Y': -3, 'V': -3},
    'H': {'A': -2, 'R': 0, 'N': 1, 'D': -1, 'C': -3, 'Q': 0, 'E': 0, 'G': -2, 'H': 8, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -1, 'P': -2, 'S': -1, 'T': -2, 'W': -2, 'Y': 2, 'V': -3},
    'I': {'A': -1, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -3, 'E': -3, 'G': -4, 'H': -3, 'I': 4, 'L': 2, 'K': -3, 'M': 1, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -3, 'Y': -1, 'V': 3},
    'L': {'A': -1, 'R': -2, 'N': -3, 'D': -4, 'C': -1, 'Q': -2, 'E': -3, 'G': -4, 'H': -3, 'I': 2, 'L': 4, 'K': -2, 'M': 2, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -2, 'Y': -1, 'V': 1},
    'K': {'A': -1, 'R': 2, 'N': 0, 'D': -1, 'C': -3, 'Q': 1, 'E': 1, 'G': -2, 'H': -1, 'I': -3, 'L': -2, 'K': 5, 'M': -1, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -3},
    'M': {'A': -1, 'R': -1, 'N': -2, 'D': -3, 'C': -1, 'Q': 0, 'E': -2, 'G': -3, 'H': -2, 'I': 1, 'L': 2, 'K': -1, 'M': 5, 'F': 0, 'P': -2, 'S': -1, 'T': -1, 'W': -1, 'Y': -1, 'V': 1},
    'F': {'A': -2, 'R': -3, 'N': -3, 'D': -3, 'C': -2, 'Q': -3, 'E': -3, 'G': -3, 'H': -1, 'I': 0, 'L': 0, 'K': -3, 'M': 0, 'F': 6, 'P': -4, 'S': -2, 'T': -2, 'W': 1, 'Y': 3, 'V': -1},
    'P': {'A': -1, 'R': -2, 'N': -2, 'D': -1, 'C': -3, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -3, 'L': -3, 'K': -1, 'M': -2, 'F': -4, 'P': 7, 'S': -1, 'T': -1, 'W': -4, 'Y': -3, 'V': -2},
    'S': {'A': 1, 'R': -1, 'N': 1, 'D': 0, 'C': -1, 'Q': 0, 'E': 0, 'G': 0, 'H': -1, 'I': -2, 'L': -2, 'K': 0, 'M': -1, 'F': -2, 'P': -1, 'S': 4, 'T': 1, 'W': -3, 'Y': -2, 'V': -2},
    'T': {'A': 0, 'R': -1, 'N': 0, 'D': -1, 'C': -1, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 5, 'W': -2, 'Y': -2, 'V': 0},
    'W': {'A': -3, 'R': -3, 'N': -4, 'D': -4, 'C': -2, 'Q': -2, 'E': -3, 'G': -2, 'H': -2, 'I': -3, 'L': -2, 'K': -3, 'M': -1, 'F': 1, 'P': -4, 'S': -3, 'T': -2, 'W': 11, 'Y': 2, 'V': -3},
    'Y': {'A': -2, 'R': -2, 'N': -2, 'D': -3, 'C': -2, 'Q': -1, 'E': -2, 'G': -3, 'H': 2, 'I': -1, 'L': -1, 'K': -2, 'M': -1, 'F': 3, 'P': -3, 'S': -2, 'T': -2, 'W': 2, 'Y': 7, 'V': -1},
    'V': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -2, 'E': -2, 'G': -3, 'H': -3, 'I': 3, 'L': 1, 'K': -3, 'M': 1, 'F': -1, 'P': -2, 'S': -2, 'T': 0, 'W': -3, 'Y': -1, 'V': 4}
}
    # Compute the score of the seed word using BLOSUM62
    wordScore = 0
    for i in range(len(word)):
        q_char = querySequence[wordQueryStartIndex + i]
        d_char = databaseSequence[wordDataStartIndex + i]
        wordScore += BLOSUM62[q_char][d_char]

    # Set an extension threshold
    extensionThreshold = 0


    alignment, score = extendAlignment(
        word,
        wordScore,
        wordQueryStartIndex,
        wordDataStartIndex,
        extensionThreshold,
        querySequence,
        databaseSequence,
        BLOSUM62
    )

    print("Query sequence:    ", querySequence)
    print("Database sequence: ", databaseSequence)
    print("Seed word:         ", word)
    print("Seed score:        ", wordScore)
    print("Extended alignment:", alignment)
    print("Extension score:   ", score)

testExtendAlignment()


Query sequence:     LIAWHCMPNAAA
Database sequence:  MQLIAWHCMPNAAAKK
Seed word:          WHC
Seed score:         28
Extended alignment: LIAWHCMPNAAA
Extension score:    70


In [ ]:
"""
Main runner method for the BLAST algorithm, groups all the functions from above into one cohesive function
querySequence: query sequence to use for the search
databaseSequence: database sequence to search into
Return: All searches that are close enough to the original query sequence
"""
def BLASTRunner(querySequence, databaseSequence, wordLength):

    # List of amino acids
    aminoAcids = ['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V']

    # Python dictionary (Hash table) containing the scoring values between two amino acids
    BLOSUM62 = {
        'A': {'A': 4, 'R': -1, 'N': -2, 'D': -2, 'C': 0, 'Q': -1, 'E': -1, 'G': 0, 'H': -2, 'I': -1, 'L': -1, 'K': -1,
              'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 0, 'W': -3, 'Y': -2, 'V': 0},
        'R': {'A': -1, 'R': 5, 'N': 0, 'D': -2, 'C': -3, 'Q': 1, 'E': 0, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 2,
              'M': -1, 'F': -3, 'P': -2, 'S': -1, 'T': -1, 'W': -3, 'Y': -2, 'V': -3},
        'N': {'A': -2, 'R': 0, 'N': 6, 'D': 1, 'C': -3, 'Q': 0, 'E': 0, 'G': 0, 'H': 1, 'I': -3, 'L': -3, 'K': 0,
              'M': -2, 'F': -3, 'P': -2, 'S': 1, 'T': 0, 'W': -4, 'Y': -2, 'V': -3},
        'D': {'A': -2, 'R': -2, 'N': 1, 'D': 6, 'C': -3, 'Q': 0, 'E': 2, 'G': -1, 'H': -1, 'I': -3, 'L': -4, 'K': -1,
              'M': -3, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -4, 'Y': -3, 'V': -3},
        'C': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': 9, 'Q': -3, 'E': -4, 'G': -3, 'H': -3, 'I': -1, 'L': -1, 'K': -3,
              'M': -1, 'F': -2, 'P': -3, 'S': -1, 'T': -1, 'W': -2, 'Y': -2, 'V': -1},
        'Q': {'A': -1, 'R': 1, 'N': 0, 'D': 0, 'C': -3, 'Q': 5, 'E': 2, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 1,
              'M': 0, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -2, 'Y': -1, 'V': -2},
        'E': {'A': -1, 'R': 0, 'N': 0, 'D': 2, 'C': -4, 'Q': 2, 'E': 5, 'G': -2, 'H': 0, 'I': -3, 'L': -3, 'K': 1,
              'M': -2, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -2},
        'G': {'A': 0, 'R': -2, 'N': 0, 'D': -1, 'C': -3, 'Q': -2, 'E': -2, 'G': 6, 'H': -2, 'I': -4, 'L': -4, 'K': -2,
              'M': -3, 'F': -3, 'P': -2, 'S': 0, 'T': -2, 'W': -2, 'Y': -3, 'V': -3},
        'H': {'A': -2, 'R': 0, 'N': 1, 'D': -1, 'C': -3, 'Q': 0, 'E': 0, 'G': -2, 'H': 8, 'I': -3, 'L': -3, 'K': -1,
              'M': -2, 'F': -1, 'P': -2, 'S': -1, 'T': -2, 'W': -2, 'Y': 2, 'V': -3},
        'I': {'A': -1, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -3, 'E': -3, 'G': -4, 'H': -3, 'I': 4, 'L': 2, 'K': -3,
              'M': 1, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -3, 'Y': -1, 'V': 3},
        'L': {'A': -1, 'R': -2, 'N': -3, 'D': -4, 'C': -1, 'Q': -2, 'E': -3, 'G': -4, 'H': -3, 'I': 2, 'L': 4, 'K': -2,
              'M': 2, 'F': 0, 'P': -3, 'S': -2, 'T': -1, 'W': -2, 'Y': -1, 'V': 1},
        'K': {'A': -1, 'R': 2, 'N': 0, 'D': -1, 'C': -3, 'Q': 1, 'E': 1, 'G': -2, 'H': -1, 'I': -3, 'L': -2, 'K': 5,
              'M': -1, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -3, 'Y': -2, 'V': -3},
        'M': {'A': -1, 'R': -1, 'N': -2, 'D': -3, 'C': -1, 'Q': 0, 'E': -2, 'G': -3, 'H': -2, 'I': 1, 'L': 2, 'K': -1,
              'M': 5, 'F': 0, 'P': -2, 'S': -1, 'T': -1, 'W': -1, 'Y': -1, 'V': 1},
        'F': {'A': -2, 'R': -3, 'N': -3, 'D': -3, 'C': -2, 'Q': -3, 'E': -3, 'G': -3, 'H': -1, 'I': 0, 'L': 0, 'K': -3,
              'M': 0, 'F': 6, 'P': -4, 'S': -2, 'T': -2, 'W': 1, 'Y': 3, 'V': -1},
        'P': {'A': -1, 'R': -2, 'N': -2, 'D': -1, 'C': -3, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -3, 'L': -3,
              'K': -1, 'M': -2, 'F': -4, 'P': 7, 'S': -1, 'T': -1, 'W': -4, 'Y': -3, 'V': -2},
        'S': {'A': 1, 'R': -1, 'N': 1, 'D': 0, 'C': -1, 'Q': 0, 'E': 0, 'G': 0, 'H': -1, 'I': -2, 'L': -2, 'K': 0,
              'M': -1, 'F': -2, 'P': -1, 'S': 4, 'T': 1, 'W': -3, 'Y': -2, 'V': -2},
        'T': {'A': 0, 'R': -1, 'N': 0, 'D': -1, 'C': -1, 'Q': -1, 'E': -1, 'G': -2, 'H': -2, 'I': -1, 'L': -1, 'K': -1,
              'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 5, 'W': -2, 'Y': -2, 'V': 0},
        'W': {'A': -3, 'R': -3, 'N': -4, 'D': -4, 'C': -2, 'Q': -2, 'E': -3, 'G': -2, 'H': -2, 'I': -3, 'L': -2,
              'K': -3, 'M': -1, 'F': 1, 'P': -4, 'S': -3, 'T': -2, 'W': 11, 'Y': 2, 'V': -3},
        'Y': {'A': -2, 'R': -2, 'N': -2, 'D': -3, 'C': -2, 'Q': -1, 'E': -2, 'G': -3, 'H': 2, 'I': -1, 'L': -1, 'K': -2,
              'M': -1, 'F': 3, 'P': -3, 'S': -2, 'T': -2, 'W': 2, 'Y': 7, 'V': -1},
        'V': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': -1, 'Q': -2, 'E': -2, 'G': -3, 'H': -3, 'I': 3, 'L': 1, 'K': -3,
              'M': 1, 'F': -1, 'P': -2, 'S': -2, 'T': 0, 'W': -3, 'Y': -1, 'V': 4}
    }

    # Threshold value, 11 is an arbitrary value used as the default in some BLAST implementations
    threshold = 11
    # Extension threshold is an artibrary value that manages the extension, and serves as an 'extension cutoff'
    extensionThreshold = 11
    # Minimum length makes sure that the approximate alignments are over a certain length, to try and filter out random hits, so only meaningful alignments remain
    minExtensionWordLength = 6

    # generate the initial words from the query sequence
    # format: (word, index)
    initialWords = generateInitialWords(querySequence, wordLength)

    # generate all the neighborhood words (words that are close to the original words, and satisfy a certain threshold)
    # neighborhoodWords format: (neighborhoodWord, (index, neighborhoodWordScore))
    # Having the indices along with the words is VERY IMPORTANT because it allows to to see where to do the extension from in the query sequence
    neighborhoodWords = {}
    for wordAndIndex in initialWords:
        tmpNeighborhoodWords = generateNeighborhoodWords(aminoAcids, BLOSUM62, wordAndIndex[1], threshold)
        # Allows for fast retrieval of indexes associated with each word
        for word in tmpNeighborhoodWords:
            neighborhoodWords[word[1]] = (wordAndIndex[0], word[0])

    approximatePatternMatches = []

    # iterate through all the neighborhood words
    for word in neighborhoodWords.keys():
        # print(word + ": " + str(neighborhoodWords[word]))
        # Find indexes where there is an exact pattern match
        initialPatternMatchesIndexes = findInitialPatternMatch(word, databaseSequence)

        # Using these indexes, calculate a pattern match in both directions of the database sequence, using the query sequence as reference
        for wordDataStartIndex in initialPatternMatchesIndexes:

            wordScore = neighborhoodWords.get(word)[1]
            wordQueryStartIndex = neighborhoodWords.get(word)[0]
            # Find the pattern match alignments and add to approximatePatternMatches
            approximatePatternMatches.append(extendAlignment(word, wordScore, wordQueryStartIndex, wordDataStartIndex, extensionThreshold, querySequence, databaseSequence, BLOSUM62))

    # for extendedAlignment in approximatePatternMatches:
    #     print(str(extendedAlignment))

    # Remove duplicates by using a dictionary
    approxMatchesNoDup = []
    for extendedWord in approximatePatternMatches:
      if extendedWord not in approxMatchesNoDup:
        approxMatchesNoDup.append(extendedWord)

    approxMatchesFinal = []
    # Remove amino acid match sequences that are shorter than a certain minimum length
    for i in range(len(approxMatchesNoDup)):
      currentWordLength = len(approxMatchesNoDup[i][0])
      if currentWordLength >= minExtensionWordLength:
        approxMatchesFinal.append(approxMatchesNoDup[i])

    return approxMatchesFinal

In [ ]:
"""
Used this for testing, not really needed
"""
def readDatabaseFile():
    database = ""
    with open("/content/database.txt", "r") as databaseFile:
        for line in databaseFile:
            database += line.rstrip('\n')

    return database

In [ ]:
"""
Main method for code execution
Currently being used for testing
"""
if __name__ == "__main__":
    # For testing purposes only
    # testGenerateInitialWords()
    # testGenerateAminoAcidSequences()
    # testGenerateNeighborhoodWords()
    # testFindInitialPatternMatch()

    querySequence = "LIAWHCMPNAAA"
    databaseSequence = getNCBIProteinDatabase()

    print("--- BLAST Algorithm Results ---")
    print(f"Query Sequence: {querySequence}")
    approximateMatches = BLASTRunner(querySequence, databaseSequence, 3)
    print("Approximate Pattern Matches:")
    for match in approximateMatches:
        print(f"{match[0]} ({match[1]})")

Raw FASTA Record: 
>pdb|9EF4|B Chain B, Insulin-like receptor
MFNMPRGVTKSKSKRGKIKMENDMAAAATTTACTLGHICVLCRQEMLLDTCCCRQAVEAVDSPASSEEAY
SSSNSSSCQASSEISAEEVWFLSHDDIVLCRRPKFDEVETTGKKRDVKCSGHQCSNECDDGSTKNNRQQR
ENFNIFSNCHNILRTLQSLLLLMFNCGIFNKRRRRQHQQQHHHHYQHHHQQHHQQHHQRQQANVSYTKFL
LLLQTLAAATTRLSLSPKNYKQQQQLQHNQQLPRATPQQKQQEKDRHKCFHYKHNYSYSPGISLLLFILL
ANTLAIQAVVLPAHQQHLLHNDIADGLDKTALSVSGTQSRWTRSESNPTMRLSQNVKPCKSMDIRNMVSH
FNQLENCTVIEGFLLIDLINDASPLNRSFPKLTEVTDYIIIYRVTGLHSLSKIFPNLSVIRGNKLFDGYA
LVVYSNFDLMDLGLHKLRSITRGGVRIEKNHKLCYDRTIDWLEILAENETQLVVLTENGKEKECRLSKCP
GEIRIEEGHDTTAIEGELNASCQLHNNRRLCWNSKLCQTKCPEKCRNNCIDEHTCCSQDCLGGCVIDKNG
NESCISCRNVSFNNICMDSCPKGYYQFDSRCVTANECITLTKFETNSVYSGIPYNGQCITHCPTGYQKSE
NKRMCEPCPGGKCDKECSSGLIDSLERAREFHGCTIITGTEPLTISIKRESGAHVMDELKYGLAAVHKIQ
SSLMVHLTYGLKSLKFFQSLTEISGDPPMDADKYALYVLDNRDLDELWGPNQTVFIRKGGVFFHFNPKLC
VSTINQLLPMLASKPKFFEKSDVGADSNGNRGSCGTAVLNVTLQSVGANSAMLNVTTKVEIGEPQKPSNA
TIVFKDPRAFIGFVFYHMIDPYGNSTKSSDDPCDDRWKVSSPEKSGVMVLSNLIPYTNYSYYVRTMAISS
ELTNAESDVKNFRTN